In [1]:
import sys
import zipfile
import os
from random import sample 

import numpy as np
import pandas as pd
from IPython.display import display, HTML

sys.path.append("../../libs/")

from datasetutil import gerar_estatiscas_df, substituir_valores_nulos , verify_gg4cc , save_zip2

## Metodos

In [2]:
def gerar_ds(set_a,set_b,n=5000,m=.5):
    '''
    Recupera um conjunto com os ids das entidades que serão utilizadas no 
    conjunto a, conjunto b e gabarito
    
    '''
    
    um = 1 - m
    
    total_m = int(n * m)
    total_um = int(n * um)
    
    inter_ab = list(set_a & set_b)
    
    nb = set_a.difference(inter_ab)
    na = set_b.difference(inter_ab)
    
    matches = sample(inter_ab, total_m)
    unmatch_a = sample(nb, total_um)
    unmatch_b = sample(na, total_um)
    
    return matches + unmatch_a, matches + unmatch_b , matches

In [3]:
def popular_ds(dsa,dsb,list_a,list_b,gab):
    '''
        Monta o dataset com base nas listas
        
        @return dataset a
        @return dataset b
        @return goldstanrd
    '''
    za = dsa[dsa.voter_id.isin(list_a)]
    zb = dsb[dsb.voter_id.isin(list_b)]

    gs = pd.DataFrame(list(zip(gab)),columns=['id1'])
    gs['id2'] = gs.id1
    return za, zb , gs

def save_all(dsa,dsb,gs,outfile):
    '''
    Salva tudo em um zip
    '''
    
    n_a = dsa.isnull().sum()
    s_a = dsa.describe(include='object')
    n_b = dsb.isnull().sum()
    s_b = dsb.describe(include='object')
    
    sa = gerar_estatiscas_df(n_a,s_a)
    sb = gerar_estatiscas_df(n_b,s_b)
    
    dfa = substituir_valores_nulos(dsa)
    dfb = substituir_valores_nulos(dsb)
    
    save_zip2(dfa,dfb,gs,sa,sb,outfile)

In [7]:
def read(zinput):
    '''
        Ler zip com os datasets - para o ano de 2014 e 2017
    '''
    zf = zipfile.ZipFile(zinput) 
    fwidths = [35,20,20,3,
               4,1,8,1+7+4+2, #mambojambo_1 (itens 8 ao 11)
               30,6+2,13,35, #mambojambo_2 (itens 13 ao 14)
               2,5,50+50+50+50+50 , #mambojambo_2 (itens 19 ao 23)
               13
              ]
    colunas = ['lastname','firstname','middlename','namesufix',
               'birthyear','gender','date_registration','mambojambo_1',
               'street_name','mambojambo_2','extension','city',
               'state','zip','mambojambo_3',
               'voter_id'
              ]


    saida = []
    for arquivo in zipfile.ZipFile.namelist(zf):
        
        df = pd.read_fwf(zf.open(arquivo),
                         header=0,encoding="iso-8859-1",
#                          nrows=15,
                         widths = fwidths,
                         names = colunas)
        
        del df['mambojambo_1']
        del df['mambojambo_2']
        del df['mambojambo_3']
        
        if len(saida) == 0:
            saida = df
        else:
            saida = pd.concat([saida,df])
            
        
    return saida

In [23]:
def alterar_colunas(df):
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]
    cols[0] = 'id'
    df.columns = cols
    return df

## Leitura dos dados

In [4]:
base_dir = 'D:'+os.sep+'Dados'+os.sep+'OneDrive'+os.sep+'Doutorado'+os.sep+'Datasets'+os.sep
ds_files = base_dir + 'MVR'+os.sep+'michiganvoters.info'+os.sep+'download'+os.sep
fa = '20140901'+os.sep+'foia_voters.zip'
fb = '20171031'+os.sep+'foia_voters.zip'
# fb = '20200302'+os.sep+'EntireStateVoter.zip'


In [8]:
a = read(ds_files + fa)
b = read(ds_files + fb)

In [9]:
sa = set(a.voter_id.unique())
sb = set(b.voter_id.unique())

In [27]:
n = 1500
    
base_dir = 'D:'+os.sep+'Dados'+os.sep+'OneDrive'+os.sep+'Doutorado'+os.sep+'workspace'+os.sep
out_files = base_dir + 'tl_pprl2'+os.sep+'datasets'+os.sep+'MVR'+os.sep


prefix = 'michiganvoters_'
sufix = '.zip'

for i in [.1]:
    la,lb , gab = gerar_ds(sa,sb,m=i,n=n)
    dsa , dsb , gs = popular_ds(a,b,la,lb,gab)
    dsa = alterar_colunas(dsa)
    dsb = alterar_colunas(dsb)
    of = out_files + prefix + str(n) +"_"+ str(i)+sufix
    save_all(dsa,dsb,gs,of)
    print(of)
    

D:\Dados\OneDrive\Doutorado\workspace\tl_pprl2\datasets\MVR\michiganvoters_1500_0.1.zip


## Scratch

In [26]:
dsb

,id,lastname,firstname,middlename,namesufix,birthyear,gender,date_registration,street_name,extension,city,state,zip
46447,575015689,BAKER,RICHARD,JAMES,,1993,M,8022017,GRAYFIELD,,REDFORD,MI,48239
53898,33346668,BRAGGS,ALEC,ROBERT,,1996,M,5312017,MOUNT HOPE,,LANSING,MI,48917
58527,33259634,BUDROW,JASON,EARL,,1984,M,4182017,TAYLOR,,CLARKSTON,MI,48346
74452,33248119,STURDIVANT,TANINA,MARIE,,1971,F,4112017,CHIPPEWA,,ROMULUS,MI,48174
75762,33541579,FITZGERALD,PAMELA,JOY,,1945,F,8172017,RIAL LAKE,,HOWELL,MI,48843
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7351174,32875405,COLLINS,JERNARD,CORDELL,,1989,M,10112016,CIVIC CENTER,STE 2113,SOUTHFIELD,MI,48076
7356692,33097376,KNAUSS,AARON,MICHAEL,,1989,M,1312017,COVE RESORT,,BRONSON,MI,49028
7357040,595001022,NUMERICK,JUSTIN,D,,1998,M,10102016,MISTY RIDGE,,MIDDLEVILLE,MI,49333
7361519,33097241,JACKSON,RUSSELL,MICHAEL,,1979,M,1312017,KINGSTON,,DEARBORN HEIGHTS,MI,48125


In [ ]:
dsb[dsb.voter_id == 108341437]

In [ ]:
dsa

['voter_id',
 'lastname',
 'firstname',
 'middlename',
 'namesufix',
 'birthyear',
 'gender',
 'date_registration',
 'street_name',
 'extension',
 'city',
 'state',
 'zip']

In [22]:
dsa

,id,lastname,firstname,middlename,namesufix,birthyear,gender,date_registration,street_name,extension,city,state,zip
18156,5023857,CARDELLA,JOANN,DOROTHY,,1937,F,10041976,BEVERLY,,ROSEVILLE,MI,48066
88577,348864,JONES,SANDRA,CAROL,,1956,F,10081984,WHITCOMB,,DETROIT,MI,48235
113801,441247,YATES,CHARMINE,CERENE,,1963,F,10102000,OAKMAN,,DETROIT,MI,48238
118070,460775,WALK,MINNIE,L,,1923,F,10102000,WILKINS,APT 1069,DETROIT,MI,48201
128777,509008,HAMILTON,ANNA,MARIA,,1963,F,1011900,BASSETT,,DETROIT,MI,48217
...,...,...,...,...,...,...,...,...,...,...,...,...,...
331731,835008129,TSERMENGAS,AMANDA,JANE,,1984,F,9122013,EUREKA,,SOUTHGATE,MI,48195
334015,108774093,NELLIS,ISAIAH,GRANT,,1983,M,5232012,MALCOLM X,,LANSING,MI,48912
351240,108763510,TALTON,DIANE,DENISE,,1956,F,7302014,SHIAWASSEE,APT 402,SOUTHFIELD,MI,48033
398872,1995011803,FARRIS,JESSICA,ANN,,1991,F,10102013,BALDWIN,APT H,JENISON,MI,49428
